In [1]:
from pennylane import numpy as np
import pennylane as qml
from pennylane import qchem
from matplotlib import pyplot as plt
from functions_vqe_crx_3 import vqe_ng
import numpy as np
from joblib import Parallel, delayed

device = 'cpu'

## He<sub>2</sub> hamiltonian definition 

In [2]:
dataset = qml.data.load('qchem', molname="He2")[0]
H, qubits = dataset.hamiltonian, len(dataset.hamiltonian.wires)
print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

Number of qubits =  8
The Hamiltonian is  -1.6743100655776797 * I([0, 1, 2, 3, 4, 5, 6, 7]) + 0.3416412951109973 * Z(0) + 0.01523407338776362 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 0.01523407338776362 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.3416408311620236 * Z(2) + 0.025441159889832372 * (Z(0) @ Z(2)) + 0.015234388911867857 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 0.015234388911867857 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 7.317214998671384e-08 * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 7.317214998671384e-08 * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.5193693204313432 * Z(4) + 0.09152839641299164 * (Z(0) @ Z(4)) + -0.5193733761290926 * Z(6) + 0.0915284611478574 * (Z(0) @ Z(6)) + 0.34164129511099733 * Z(1) + 0.14108387203893755 * (Z(0) @ Z(1)) + 0.03956129782205163 * (Y(0) @ Z(2) @ Z(3) @ Y(4)) + 0.03956129782205163 * (X(0) @ Z(2) @ Z(3) @ X(4)) + 0.11564281616296 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.11564281616296 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.11564281616296 * (X(0) @ X(1) @

In [3]:
dev = qml.device("lightning.qubit", wires=qubits)

In [4]:
electrons = 4
hf_state = qml.qchem.hf_state(electrons, qubits)
singles, doubles = qchem.excitations(electrons, qubits)
print(hf_state)

[1 1 1 1 0 0 0 0]


In [5]:
s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

## Distribution of results

### Parameters

In [16]:
max_iterations = 200

layers = 20

n_runs = 50

conv_tol = 1e-06

weights_lr = 0.005
phi_lr = 0.07

thetas = np.linspace(0, 25, 12, dtype=int)
n_theta = len(thetas)

max_workers = 21

In [17]:
prefix1 = "../local_data/he2_log1/"
spec1 = "_crx_1"

prefix2 = prefix1
spec2 = "_crx_2"

In [18]:
import json

run_info = {
    "max_iterations": max_iterations,
    "layers": layers,
    "n_runs": n_runs,
    "conv_tol": conv_tol,
    "weights_lr": weights_lr,
    "phi_lr": phi_lr,
    "prefix1": prefix1,
    "spec1": spec1,
    "prefix2": prefix2,
    "spec2": spec2,
    "device": device,
    "qubits": qubits,
    "thetas": thetas.tolist(),
    "n_theta": n_theta,
    "opt": "SDG",
    "ansatz": "cascade with CRX gates",
}

with open(prefix1 + "run_info.txt", "w") as f:
    json.dump(run_info, f, indent=4)

In [19]:
def run_single_simulation_2(i, theta, j):
    """
    Worker function that runs one iteration.
    Arguments are passed directly, not as a tuple.
    """
    energy, grad_norm, grad_var, conv = vqe_ng(H, qubits, layers, theta=theta, weights_lr=weights_lr, phi_lr=phi_lr, max_iterations=max_iterations, conv_tol=1e-06, verbose=False)

    return i, j, (energy, grad_norm, grad_var, conv)

### NG

In [20]:
tasks = (delayed(run_single_simulation_2)(i, theta, j)
         for i, theta in enumerate(thetas)
         for j in range(n_runs))
print('number of tasks:', n_theta * n_runs)
results = Parallel(n_jobs=max_workers, verbose=11)(tasks)

number of tasks: 600


[Parallel(n_jobs=21)]: Using backend LokyBackend with 21 concurrent workers.
[Parallel(n_jobs=21)]: Done   1 tasks      | elapsed:   24.3s
[Parallel(n_jobs=21)]: Done   2 tasks      | elapsed:   24.5s
[Parallel(n_jobs=21)]: Done   3 tasks      | elapsed:   24.9s
[Parallel(n_jobs=21)]: Done   4 tasks      | elapsed:   30.1s
[Parallel(n_jobs=21)]: Done   5 tasks      | elapsed:   31.0s
[Parallel(n_jobs=21)]: Done   6 tasks      | elapsed:   31.2s
[Parallel(n_jobs=21)]: Done   7 tasks      | elapsed:   31.8s
[Parallel(n_jobs=21)]: Done   8 tasks      | elapsed:   31.9s
[Parallel(n_jobs=21)]: Done   9 tasks      | elapsed:   32.4s
[Parallel(n_jobs=21)]: Done  10 tasks      | elapsed:   32.4s
[Parallel(n_jobs=21)]: Done  11 tasks      | elapsed:   32.5s
[Parallel(n_jobs=21)]: Done  12 tasks      | elapsed:   32.6s
[Parallel(n_jobs=21)]: Done  13 tasks      | elapsed:   32.6s
[Parallel(n_jobs=21)]: Done  14 tasks      | elapsed:   32.9s
[Parallel(n_jobs=21)]: Done  15 tasks      | elapsed:  

#### Save

In [21]:
# 1. Initialize result arrays (same as before)
energies = np.zeros((n_theta, n_runs, max_iterations+1))
grad_norms = np.zeros((n_theta, n_runs, max_iterations))
grad_variances = np.zeros((n_theta, n_runs, max_iterations))
convergences = np.zeros((n_theta, n_runs), dtype=bool)

for i, j, result in results:
    energies[i, j, :] = np.array(result[0])
    grad_norms[i, j, :] = np.array(result[1])
    grad_variances[i, j, :] = np.array(result[2])
    convergences[i, j] = result[3]

In [22]:
np.save(prefix2+"energies"+spec2+".npy", energies)
np.save(prefix2+"grad_norms"+spec2+".npy", grad_norms)
np.save(prefix2+"grad_variances"+spec2+".npy", grad_variances)
np.save(prefix2+"convergences"+spec2+".npy",convergences)